In [1]:
#download cifar 10 python version
!pip install torch torchvision matplotlib numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

In [3]:
%mkdir dataset

In [4]:
import requests
import os
import time

def download_file_with_progress(url, save_path):
    """下载文件并显示进度"""
    try:
        # 发送HEAD请求获取文件大小
        response_head = requests.head(url, allow_redirects=True)
        file_size = int(response_head.headers.get('Content-Length', 0))

        # 发送GET请求开始下载，stream模式分块读取
        response = requests.get(url, stream=True, allow_redirects=True)
        response.raise_for_status()  # 检查请求是否成功

        # 创建保存目录（如果不存在）
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        # 分块写入文件并显示进度
        downloaded_size = 0
        chunk_size = 8192  # 每次读取8KB
        start_time = time.time()

        print(f"开始下载: {url}")
        print(f"文件大小: {file_size / (1024 * 1024):.2f} MB")

        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
                    downloaded_size += len(chunk)

                    # 计算并显示下载进度
                    progress = downloaded_size / file_size * 100
                    speed = downloaded_size / (time.time() - start_time) / 1024
                    print(f"\r进度: {progress:.2f}% | 已下载: {downloaded_size / (1024 * 1024):.2f} MB | 速度: {speed:.2f} KB/s", end='')

        print("\n下载完成!")
        return True

    except requests.exceptions.RequestException as e:
        print(f"下载失败: {e}")
        return False
    except Exception as e:
        print(f"发生错误: {e}")
        return False

# 使用示例
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
save_path = "./dataset/cifar-10-python.tar.gz"  # 自定义保存路径
download_file_with_progress(url, save_path)

开始下载: https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
文件大小: 162.60 MB
进度: 100.00% | 已下载: 162.60 MB | 速度: 27039.63 KB/s
下载完成!


True

In [5]:
import tarfile

def extract_tar(file_path, extract_dir):
    with tarfile.open(file_path, 'r:gz') as tar:
        tar.extractall(path=extract_dir)
    print(f"解压完成，文件位于: {extract_dir}")

extract_tar("./dataset/cifar-10-python.tar.gz", "./cifar10_data/")

解压完成，文件位于: ./cifar10_data/


In [6]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [7]:
label_name=["airplane",
            "automobile",
            "bird",
            "cat",
            "deer",
            "dog",
            "frog",
            "horse",
            "ship",
            "truck"
            ]

In [8]:
import glob
train_list=glob.glob("/content/cifar10_data/cifar-10-batches-py/data_batch_*")
train_list

['/content/cifar10_data/cifar-10-batches-py/data_batch_2',
 '/content/cifar10_data/cifar-10-batches-py/data_batch_1',
 '/content/cifar10_data/cifar-10-batches-py/data_batch_5',
 '/content/cifar10_data/cifar-10-batches-py/data_batch_3',
 '/content/cifar10_data/cifar-10-batches-py/data_batch_4']

In [9]:

test_list=glob.glob("/content/cifar10_data/cifar-10-batches-py/test_batch")
test_list

['/content/cifar10_data/cifar-10-batches-py/test_batch']

In [10]:
for l in train_list:
    print(l)
    l_dict=unpickle(l)
    print(l_dict.keys())


/content/cifar10_data/cifar-10-batches-py/data_batch_2
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
/content/cifar10_data/cifar-10-batches-py/data_batch_1
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
/content/cifar10_data/cifar-10-batches-py/data_batch_5
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
/content/cifar10_data/cifar-10-batches-py/data_batch_3
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
/content/cifar10_data/cifar-10-batches-py/data_batch_4
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])


In [11]:
%mkdir ./cifar10_data/TRAIN

In [12]:
import os
save_path="./cifar10_data/TRAIN"
for im_idx,im_data in enumerate(l_dict[b'data']):
  # print(im_idx)
  # print(im_data)
  im_label=l_dict[b'labels'][im_idx]
  im_name=l_dict[b'filenames'][im_idx]
  # print(im_label)
  # print(im_name)
  print(im_label,im_name,im_data)
  im_label_name=label_name[im_label]
  im_data=np.reshape(im_data,[3,32,32])
  im_data=np.transpose(im_data,[1,2,0])
  # cv2_imshow(im_data,cv2.resize(im_data,(200,200)))

  if not os.path.exists("{}/{}".format(save_path,im_label_name)):
    os.mkdir("{}/{}".format(save_path,im_label_name))
  cv2.imwrite("{}/{}/{}".format(save_path,im_label_name,im_name.decode("utf-8")),im_data)


Streaming output truncated to the last 5000 lines.
5 b'lapdog_s_002196.png' [ 66  70  73 ... 174 162 202]
6 b'frog_s_002027.png' [203 199 200 ... 127 127 127]
7 b'dawn_horse_s_001542.png' [247 247 247 ...  30  24  16]
7 b'dawn_horse_s_001974.png' [130 145 134 ...  95  97  97]
1 b'estate_car_s_000994.png' [ 99 147 187 ... 128 118  93]
7 b'dawn_horse_s_000965.png' [126 127 129 ...  60  58  65]
9 b'dump_truck_s_001908.png' [143 144 145 ...  64  62 108]
3 b'tabby_cat_s_000946.png' [ 53  67  42 ...  61  60 105]
7 b'arabian_s_002060.png' [ 62  96 114 ... 100  89  81]
3 b'tabby_cat_s_000602.png' [225 214 207 ...  38  36  33]
2 b'anthus_pratensis_s_000222.png' [ 91  97 102 ... 108 105 103]
1 b'automobile_s_002152.png' [210 212 212 ...   1   2   2]
1 b'convertible_s_001097.png' [189 195 199 ... 107 105 108]
8 b'ferry_s_000158.png' [183 180 180 ... 138 144 155]
8 b'hospital_ship_s_001935.png' [159 156 157 ... 148 146 149]
5 b'pekinese_s_000059.png' [58 56 54 ... 15  8  8]
3 b'tabby_s_001431.png'

In [13]:
for l in test_list:
    print(l)
    l_dict_test=unpickle(l)
    print(l_dict_test.keys())

/content/cifar10_data/cifar-10-batches-py/test_batch
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])


In [14]:
save_path="./cifar10_data/TEST"
try:
    os.mkdir(save_path)
    print("Folder %s created!" % save_path)
except FileExistsError:
    print("Folder %s already exists" % save_path)
for im_idx,im_data in enumerate(l_dict_test[b'data']):
  # print(im_idx)
  # print(im_data)
  im_label=l_dict_test[b'labels'][im_idx]
  im_name=l_dict_test[b'filenames'][im_idx]
  # print(im_label)
  # print(im_name)
  print(im_label,im_name,im_data)
  im_label_name=label_name[im_label]
  im_data=np.reshape(im_data,[3,32,32])
  im_data=np.transpose(im_data,[1,2,0])
  # cv2_imshow(im_data,cv2.resize(im_data,(200,200)))

  if not os.path.exists("{}/{}".format(save_path,im_label_name)):
    os.mkdir("{}/{}".format(save_path,im_label_name))
  cv2.imwrite("{}/{}/{}".format(save_path,im_label_name,im_name.decode("utf-8")),im_data)

Streaming output truncated to the last 5000 lines.
7 b'lipizzan_s_002053.png' [ 51  78  71 ... 164 145 160]
6 b'grass_frog_s_001452.png' [165 139 129 ...  24  27  43]
8 b'oil_tanker_s_000444.png' [149 149 150 ...  68  73  79]
4 b'sambar_s_000048.png' [243 226 215 ... 104 133 183]
4 b'red_deer_s_001938.png' [78 87 83 ... 88 67 37]
5 b'maltese_s_002064.png' [49 54 59 ... 47 39 36]
7 b'arabian_s_002385.png' [103 107 107 ...  16  11  13]
3 b'mouser_s_001259.png' [ 11  11  10 ... 102  98  93]
4 b'dama_dama_s_000246.png' [ 20  19  17 ... 112 113 118]
7 b'dawn_horse_s_002044.png' [197 212 212 ... 111 106  96]
7 b'arabian_s_001908.png' [ 65  99 128 ...  86  78  78]
3 b'domestic_cat_s_002103.png' [27 34 42 ... 54 50 48]
6 b'leopard_frog_s_001196.png' [125 126 123 ...  64  72  75]
0 b'stealth_bomber_s_000169.png' [170 168 170 ... 121 131 146]
7 b'arabian_s_001530.png' [241 224 227 ... 219 218 220]
2 b'alauda_arvensis_s_000100.png' [133 156 162 ... 116 106 102]
6 b'american_toad_s_000785.png' [25

In [15]:
label_dict={}
for idx,name in enumerate(label_name):
  label_dict[name]=idx
label_dict

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [16]:
def default_loader(path):
  return Image.open(path).convert("RGB")

In [17]:
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
train_transform=transforms.Compose([
    transforms.RandomResizedCrop((28,28)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(90),
    transforms.RandomGrayscale(0.1),
    transforms.ColorJitter(0.3,0.3,0.3,0.3),
    transforms.ToTensor()
])

In [18]:

class MyDataset(Dataset):
  def __init__(self,im_list,transforms=None,loader=default_loader):
    super(MyDataset,self).__init__()
    imgs=[]
    for im_item in im_list:
      im_label_name=im_item.split("/")[-2]
      imgs.append([im_item,label_dict[im_label_name]])
    self.imgs=imgs
    self.transforms=transforms
    self.loader=loader
  def __getitem__(self,index):
    im_path,im_label=self.imgs[index]
    im_data=self.loader(im_path)
    if self.transforms is not null:
      im_data=self.transforms(im_data)
    return im_data,im_label
  def __len__(self):
    return len(self.imgs)

In [19]:
im_train_list=glob.glob("./cifar10_data/TEST/*/*.png")
im_test_list=glob.glob("./cifar10_data/TRAIN/*/*.png")

In [20]:
train_dataset=MyDataset(im_train_list,transforms=train_transform)
test_dataset=MyDataset(im_train_list,transforms=transforms.ToTensor())

In [21]:
train_data_loader=DataLoader(dataset=train_dataset,
            batch_size=6,
            shuffle=True,
            num_workers=4)

test_data_loader=DataLoader(dataset=test_dataset,
            batch_size=6,
            shuffle=False,
            num_workers=4)

print("number_of_train",len(train_dataset))
print("number_of_test",len(test_dataset))

number_of_train 10000
number_of_test 10000


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
